In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import re
from datetime import datetime
datetime.strptime('1 Jun 2005  1:33PM', '%d %b %Y %I:%M%p')

lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday() 


# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

odata_df = pd.read_csv('C:\\Users\\jduan\\Downloads\\strike.csv')
combine = [odata_df]

odata_df.shape

odata_df = odata_df.drop(['Closed Date Time','Geolocation (Latitude)','Geolocation (Longitude)','Incident: Incident Name'], axis=1)
odata_df.head(2)


# daymap = {0:'Monday', 1:'Tuesday',2:'Wednesday', 3:'Thursday',4:'Friday',5:'Saturday',6: 'Sunday'}
odata_df['Created Weekday']=  odata_df['Created Date Time'].apply(lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday()).astype(int)
# odata_df['Created Weekday'] = odata_df['Created Weekday'].map(daymap)

odata_df['Occurrence Weekday']=  odata_df['Occurrence Date'].dropna().apply(lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday()).astype(int)
# odata_df['Occurrence Weekday'] = odata_df['Occurrence Weekday'].map(daymap)

odata_df['Expiration Weekday']=  odata_df['Expiration Date'].dropna().apply(lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday()).astype(int)
# odata_df['Expiration Weekday'] = odata_df['Expiration Weekday'].map(daymap)

odata_df.tail(10)




odata_df.loc[odata_df['City'] == 'Nationwide', 'Area'] = odata_df.loc[odata_df['City'] == 'Nationwide', 'Area'].fillna(2)
odata_df['Area'] = odata_df['Area'].replace(np.nan, 0)

odata_df[['Area','City']].head(10)


odata_df = odata_df.drop(odata_df[(odata_df['Source complete'] == 'Manual')].index)
odata_df['Source complete'] = odata_df['Source complete'].replace(np.nan, 'R360')
odata_df['Source complete'] = odata_df['Source complete'].replace(['NC4 AB','NC4 DHL','NC4 GFP','NC4 NIMC','NIMC' ],'NC4')
odata_df['Source complete'] = odata_df['Source complete'].replace('Anvil','ANVIL')

odata_df['Source complete'].describe()

odata_df[['Source complete', 'Area']].groupby(['Source complete'], as_index=False).mean()
odata_df[['Source complete', 'Severity Number']].groupby(['Source complete'], as_index=False).mean()



#odata_df = pd.concat([odata_df,odata_df['Categories'].str.split(',').apply(pd.Series)],axis=1)
odata_df['Strike Category'] = odata_df['Categories'].apply(lambda x: [a for a in x.split(',') if "strike" in a.lower()][0].strip()) 
odata_df = odata_df.drop(['Categories'], axis=1)


odata_df[['Strike Category', 'Severity Number']].groupby(['Strike Category'], as_index=False).mean()


odata_df.tail(10)

import pycountry

countrymapping = {country.name: country.alpha_2 for country in pycountry.countries}
odata_df['Country ISO'] = odata_df['Country'].map(countrymapping)


countrymap = {"Democratic People's Republic of Korea":'KP','Curacao':'CW','The former Yugoslav Republic of Macedonia':'MK','Taiwan':'TW','United Republic of Tanzania':'TZ','Kosovo':'XK','Bahamas, The':'BS','Republic of Moldova':'MD','State of Palestine':'PS', 'Republic of Korea':'KR','Venezuela (Bolivarian Republic of)':'VE', 'Democratic Republic of the Congo':'CD','Bolivia (Plurinational State of)':'BO','Russia':'RU','Czech Republic':'CZ','Iran (Islamic Republic of)':'IR'}
odata_df['Country ISO'] = odata_df['Country ISO'].fillna(odata_df['Country'].map(countrymap))

odata_df = odata_df.drop(odata_df[(odata_df['Country ISO'].isnull())].index)


odata_df = odata_df.drop(['Country','City'], axis=1)


odata_df['Created Date Time'] = pd.to_datetime(odata_df['Created Date Time']).dt.date
odata_df['Occurrence Date'] = pd.to_datetime(odata_df['Occurrence Date']).dt.date
odata_df['Expiration Date'] = pd.to_datetime(odata_df['Expiration Date']).dt.date




odata_df['Strike Month'] = odata_df['Current Cleaned Headline'].str.findall(r'\s(Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December[w.-]+)\s')
odata_df['Strike Weekday'] = odata_df['Current Cleaned Headline'].str.findall(r'Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday')
#daymap = {'Monday':0,'Tuesday':1,'Wednesday':2, 'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}

#odata_df['Strike Weekday'] = odata_df['Strike Weekday'].map(lambda x: ".".join(x)) 

#odata_df['Strike Weekday'] = odata_df['Strike Weekday'].map(daymap)
odata_df.head(15)

import spacy


